In [1]:
#%
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tqdm


In [7]:
#%
file_path = './category.csv'
df = pd.read_csv(file_path, encoding='UTF8')
df = df.set_index(keys='CODE')
df

,BIG,MIDDLE,SMALL
CODE,,,
hb011,요식/유흥,요식,한식
hb012,요식/유흥,요식,일식/중식/양식
hb013,요식/유흥,요식,제과점
hb014,요식/유흥,요식,커피전문점
hb015,요식/유흥,요식,패스트푸드
hb016,요식/유흥,유흥,노래방
hb017,요식/유흥,유흥,기타유흥업소
hb018,요식/유흥,유흥,유흥주점
hb021,유통,유통,백화점


In [3]:
#%
from gensim import models


ko_model = models.fasttext.load_facebook_model("C:/Users/pooh5/Downloads/cc.ko.300-001.bin")
#%
for w, sim in ko_model.wv.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')
    
#%
ko_model.wv.vectors.shape


#%
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)



Python: 0.5650615692138672
자이썬: 0.5624369382858276
레일스: 0.5598082542419434
파이썬을: 0.5595802068710327
언어용: 0.5288202166557312
파이썬의: 0.5250024795532227
프로그래밍: 0.5225088596343994
wxPython: 0.5222088694572449
파이썬이나: 0.5201171636581421
함수형: 0.5187377333641052


In [8]:
#%
# 단어들 간에 유사도 확인
words = df['SMALL'].tolist()[:]
word_embedding = []
for w in words:
    print('편의점-',str(w),': ',ko_model.wv.similarity('편의점', w))
    word_embedding.append(ko_model.wv.similarity('편의점', w))
df['embedding'] = word_embedding
df.head()

편의점- 한식 :  0.1789216
편의점- 일식/중식/양식 :  0.21252921
편의점- 제과점 :  0.43229064
편의점- 커피전문점 :  0.4395384
편의점- 패스트푸드 :  0.3720898
편의점- 노래방 :  0.3514606
편의점- 기타유흥업소 :  0.22546573
편의점- 유흥주점 :  0.229173
편의점- 백화점 :  0.412799
편의점- 할인점 :  0.42538986
편의점- 슈퍼마켓 :  0.50149035
편의점- 생활잡화 :  0.29749382
편의점- 편의점 :  1.0
편의점- 정육점 :  0.35306704
편의점- 농수산물 :  0.27805534
편의점- 음/식료품 :  0.15072282
편의점- 의복/의류 :  0.12503007
편의점- 패션/잡화 :  0.14689642
편의점- 안경 :  0.19381568
편의점- 귀금속 :  0.1361093
편의점- 실내골프/헬스 :  0.16560361
편의점- 실외골프/스키 :  0.21021652
편의점- 스포츠시설 :  0.30218038
편의점- 종합레저시설 :  0.20449206
편의점- 영화/공연 :  0.13615361
편의점- 취미/오락 :  0.12304584
편의점- 서점 :  0.30448502
편의점- 호텔/콘도 :  0.22918952
편의점- 모텔/기타숙박 :  0.111040644
편의점- 여행사 :  0.26745057
편의점- 면세점 :  0.36662754
편의점- 미용서비스 :  0.2949999
편의점- 미용실 :  0.39978093
편의점- 화장품 :  0.30081877
편의점- 생활서비스 :  0.33016083
편의점- 세탁소 :  0.3773235
편의점- 업무서비스 :  0.20156215
편의점- 학원 :  0.22686931
편의점- 유아교육 :  0.10065789
편의점- 교육용품 :  0.21057644
편의점- 종합병원 :  0.22989877
편의점- 일반병원 :  0.18130638


,BIG,MIDDLE,SMALL,embedding
CODE,,,,
hb011,요식/유흥,요식,한식,0.178922
hb012,요식/유흥,요식,일식/중식/양식,0.212529
hb013,요식/유흥,요식,제과점,0.432291
hb014,요식/유흥,요식,커피전문점,0.439538
hb015,요식/유흥,요식,패스트푸드,0.372090


In [12]:

# k-means 클러스터링, df로 함. category.csv 데이터
from sklearn.cluster import KMeans

n_clusters = 5
# X = mode_card_df[['embedding_x_mean', 'embedding_y_mean', '거래금액']].values
# X = mode_card_df[['embedding_mode']].values
X = df['embedding'].values.reshape(-1,1)


kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_
# mode_card_df['Cluster'] = cluster_labels
df['Cluster'] = cluster_labels

print(mode_card_df)


                     카드번호        업종  embedding_mode
0     2568-9238-5337-7494        한식        0.178922
1     8568-8237-7432-8179  일식/중식/양식        0.212529
2     2121-8556-5241-2103        한식        0.178922
3     3646-4691-7628-1826        한식        0.178922
4     5134-8145-7073-3081        한식        0.178922
...                   ...       ...             ...
1996  7951-7952-4196-9188        한식        0.178922
1997  7393-9660-3356-1638        한식        0.178922
1998  1702-2042-3914-5877        한식        0.178922
1999  6819-5411-9779-2630  일식/중식/양식        0.212529
2000  6630-6882-1494-9726      유흥주점        0.178922

[2001 rows x 3 columns]


In [13]:
s = 0
cluster_elements = []
for i in range(n_clusters):
    print(df[df['Cluster']==i]['SMALL'].unique())
    cluster_elements.append(df[df['Cluster']==i]['SMALL'].unique().tolist())
    s = s + len(df[df['Cluster']==i]['SMALL'].unique())

print(s)
print(len(df))
print(cluster_elements)

['음/식료품' '의복/의류' '패션/잡화' '귀금속' '실내골프/헬스' '영화/공연' '취미/오락' '모텔/기타숙박' '유아교육'
 '가구' '가전/가구']
['생활잡화' '농수산물' '스포츠시설' '서점' '여행사' '미용서비스' '화장품' '생활서비스' '가전']
['편의점']
['제과점' '커피전문점' '패스트푸드' '노래방' '백화점' '할인점' '슈퍼마켓' '정육점' '면세점' '미용실' '세탁소'
 '약국' '주유소']
['한식' '일식/중식/양식' '기타유흥업소' '유흥주점' '안경' '실외골프/스키' '종합레저시설' '호텔/콘도' '업무서비스'
 '학원' '교육용품' '종합병원' '일반병원' '치과병원' '한의원' '자동차서비스' '자동차용품' 'LPG']
52
52
[['음/식료품', '의복/의류', '패션/잡화', '귀금속', '실내골프/헬스', '영화/공연', '취미/오락', '모텔/기타숙박', '유아교육', '가구', '가전/가구'], ['생활잡화', '농수산물', '스포츠시설', '서점', '여행사', '미용서비스', '화장품', '생활서비스', '가전'], ['편의점'], ['제과점', '커피전문점', '패스트푸드', '노래방', '백화점', '할인점', '슈퍼마켓', '정육점', '면세점', '미용실', '세탁소', '약국', '주유소'], ['한식', '일식/중식/양식', '기타유흥업소', '유흥주점', '안경', '실외골프/스키', '종합레저시설', '호텔/콘도', '업무서비스', '학원', '교육용품', '종합병원', '일반병원', '치과병원', '한의원', '자동차서비스', '자동차용품', 'LPG']]


In [14]:
mode_card_df_per_clusters = []
s = 0
for i in range(n_clusters):
    # print(mode_card_df[mode_card_df['업종'].isin(cluster_elements[i])])
    print(len(mode_card_df[mode_card_df['업종'].isin(cluster_elements[i])]))
    mode_card_df_per_clusters.append(mode_card_df[mode_card_df['업종'].isin(cluster_elements[i])])
    s = s + len(mode_card_df[mode_card_df['업종'].isin(cluster_elements[i])])
print(s)

91
8
38
128
1736
2001


In [15]:
#%
print(card_df.head())
print(mode_card_df.head())
print(df.head())

#%

add_columns_to_card_df = df.loc[card_df['category_id'], ['SMALL', 'embedding', 'Cluster'] ]
print(add_columns_to_card_df)
print(card_df)
#%
add_columns_to_card_df = add_columns_to_card_df.reset_index(drop=True)
#%
# minjung.csv 에 업종, 업종임베딩, 클러스터 컬럼 추가
card_df['업종'] = add_columns_to_card_df['SMALL']
card_df['embedding'] = add_columns_to_card_df['embedding']
card_df['Cluster'] = add_columns_to_card_df['Cluster']

print(card_df)

   Unnamed: 0  Card His ID                 카드번호 category_id  거래지역    상점명  \
0           0          269  2568-9238-5337-7494       hb053  충청북도    남김권   
1           1          271  2568-9238-5337-7494       hb073  충청북도  (유) 최   
2           2          273  2568-9238-5337-7494       hb022  충청북도     류이   
3           3          275  2568-9238-5337-7494       hb016  충청북도     김이   
4           4          277  2568-9238-5337-7494       hb071  충청북도  (유) 이   

         거래날짜      거래시간   거래금액  
0  2023-04-22  19:02:00  98000  
1  2023-03-19  19:04:00  57000  
2  2023-05-04  19:28:00   6500  
3  2023-04-20  20:47:00  10000  
4  2023-05-31  17:06:00  25000  
                  카드번호        업종  embedding_mode
0  2568-9238-5337-7494        한식        0.178922
1  8568-8237-7432-8179  일식/중식/양식        0.212529
2  2121-8556-5241-2103        한식        0.178922
3  3646-4691-7628-1826        한식        0.178922
4  5134-8145-7073-3081        한식        0.178922
         BIG MIDDLE     SMALL  embedding  Cluster
C

In [20]:
card_df

,Unnamed: 0,Card His ID,카드번호,category_id,거래지역,상점명,거래날짜,거래시간,거래금액,업종,embedding,Cluster
0,0,269,2568-9238-5337-7494,hb053,충청북도,남김권,2023-04-22,19:02:00,98000,스포츠시설,0.302180,1
1,1,271,2568-9238-5337-7494,hb073,충청북도,(유) 최,2023-03-19,19:04:00,57000,화장품,0.300819,1
2,2,273,2568-9238-5337-7494,hb022,충청북도,류이,2023-05-04,19:28:00,6500,할인점,0.425390,3
3,3,275,2568-9238-5337-7494,hb016,충청북도,김이,2023-04-20,20:47:00,10000,노래방,0.351461,3
4,4,277,2568-9238-5337-7494,hb071,충청북도,(유) 이,2023-05-31,17:06:00,25000,미용서비스,0.295000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
600295,600295,1142267,8400-4787-7962-6317,hb041,충청남도,주식회사 김류김,2023-05-30,14:27:00,66000,의복/의류,0.125030,0
600296,600296,1142269,8400-4787-7962-6317,hb024,충청남도,김강박,2023-04-21,10:21:00,57000,생활잡화,0.297494,1
600297,600297,1142271,8400-4787-7962-6317,hb023,충청남도,남김,2023-03-26,20:17:00,21000,슈퍼마켓,0.501490,3
600298,600298,1142273,8400-4787-7962-6317,hb011,충청남도,강박류,2023-03-19,16:36:00,46000,한식,0.178922,4


In [31]:
print(len(card_df[card_df['Cluster']==0]['카드번호'].unique()))
print(len(card_df[card_df['Cluster']==1]['카드번호'].unique()))
print(len(card_df[card_df['Cluster']==2]['카드번호'].unique()))
print(len(card_df[card_df['Cluster']==3]['카드번호'].unique()))
print(len(card_df[card_df['Cluster']==4]['카드번호'].unique()))

2001
2001
1995
2001
2001


In [32]:
print(card_df)

characters_to_remove = '-'
# card_df['거래날짜'] = card_df['거래날짜'].astype(str)
card_df['거래날짜'] = card_df['거래날짜'].str.replace(characters_to_remove, '')
characters_to_remove = ':'
card_df['거래시간'] = card_df['거래시간'].str.replace(characters_to_remove, '')
card_df['시간_numerical'] = card_df['거래날짜'] + card_df['거래시간']
print(card_df)


        Unnamed: 0  Card His ID                 카드번호 category_id  거래지역  \
0                0          269  2568-9238-5337-7494       hb053  충청북도   
1                1          271  2568-9238-5337-7494       hb073  충청북도   
2                2          273  2568-9238-5337-7494       hb022  충청북도   
3                3          275  2568-9238-5337-7494       hb016  충청북도   
4                4          277  2568-9238-5337-7494       hb071  충청북도   
...            ...          ...                  ...         ...   ...   
600295      600295      1142267  8400-4787-7962-6317       hb041  충청남도   
600296      600296      1142269  8400-4787-7962-6317       hb024  충청남도   
600297      600297      1142271  8400-4787-7962-6317       hb023  충청남도   
600298      600298      1142273  8400-4787-7962-6317       hb011  충청남도   
600299      600299      1142275  8400-4787-7962-6317       hb021  충청남도   

             상점명        거래날짜      거래시간   거래금액     업종  embedding  Cluster  
0            남김권  2023-04-22  19:02:

In [9]:

file_path = './minjeong.csv'
card_df = pd.read_csv(file_path, encoding='UTF8')
print(card_df.head())
print(len(card_df))


print(card_df['카드번호'].unique())
print(len(card_df['카드번호'].unique()))

card_IDs = card_df['카드번호'].unique()

# mean_card_df = {'카드번호': [], 'embedding_mod' : [], '거래금액': []}
mode_card_df = {'카드번호': [], '업종' : [], 'embedding_mode' : []}
# add_columns_to_card_df = {'업종_mode' : [], '업종' : [], 'cluster' : []}

for i, card_ID in tqdm.tqdm(enumerate(card_IDs)):
    '''
    _df : 카드 거래 내역 중 카드번호 같은 것 끼리 모은 임시 df
    mean_card_df : 카드번호 같은 것들 중, 최빈값 업종을 뽑아서 대표 업종으로 만든 df
    '''
    _df = card_df[card_df['카드번호']==card_ID][['category_id', '거래금액']]
    
    # print(_df['category_id'].tolist())
    # print(df.head())
    # print(df.loc[['hb011','hb012']])
    # print(df.loc[_df['category_id']]['embedding_x'].mean())
    mode_card_df['카드번호'].append(card_ID)
    mode_card_df['업종'].append(df.loc[_df['category_id']]['SMALL'].mode().iloc[0])
    mode_card_df['embedding_mode'].append(df.loc[_df['category_id']]['embedding'].mode().iloc[0])
    
    # print(df.loc[_df['category_id']]['embedding'].mode().iloc[0])
    # mean_card_df['embedding_y_mean'].append(df.loc[_df['category_id']]['embedding_y'].mean())
    # mean_card_df['거래금액'].append(_df['거래금액'].mean())
    # print(mean_card_df)

mode_card_df = pd.DataFrame(mode_card_df)
print(mode_card_df)

   Unnamed: 0  Card His ID                 카드번호 category_id  거래지역    상점명  \
0           0          269  2568-9238-5337-7494       hb053  충청북도    남김권   
1           1          271  2568-9238-5337-7494       hb073  충청북도  (유) 최   
2           2          273  2568-9238-5337-7494       hb022  충청북도     류이   
3           3          275  2568-9238-5337-7494       hb016  충청북도     김이   
4           4          277  2568-9238-5337-7494       hb071  충청북도  (유) 이   

         거래날짜      거래시간   거래금액  
0  2023-04-22  19:02:00  98000  
1  2023-03-19  19:04:00  57000  
2  2023-05-04  19:28:00   6500  
3  2023-04-20  20:47:00  10000  
4  2023-05-31  17:06:00  25000  
600300
['2568-9238-5337-7494' '8568-8237-7432-8179' '2121-8556-5241-2103' ...
 '1702-2042-3914-5877' '6819-5411-9779-2630' '6630-6882-1494-9726']
2001


2001it [00:58, 34.13it/s]

                     카드번호        업종  embedding_mode
0     2568-9238-5337-7494        한식        0.178922
1     8568-8237-7432-8179  일식/중식/양식        0.212529
2     2121-8556-5241-2103        한식        0.178922
3     3646-4691-7628-1826        한식        0.178922
4     5134-8145-7073-3081        한식        0.178922
...                   ...       ...             ...
1996  7951-7952-4196-9188        한식        0.178922
1997  7393-9660-3356-1638        한식        0.178922
1998  1702-2042-3914-5877        한식        0.178922
1999  6819-5411-9779-2630  일식/중식/양식        0.212529
2000  6630-6882-1494-9726      유흥주점        0.178922

[2001 rows x 3 columns]


In [10]:
print(mode_card_df.head())


#%
# 임베딩 벡터와 카드 번호 별로 저장
mode_card_df.to_csv('./SMALL_embedding_mode.csv') 
#%
df.head()


                  카드번호        업종  embedding_mode
0  2568-9238-5337-7494        한식        0.178922
1  8568-8237-7432-8179  일식/중식/양식        0.212529
2  2121-8556-5241-2103        한식        0.178922
3  3646-4691-7628-1826        한식        0.178922
4  5134-8145-7073-3081        한식        0.178922


,BIG,MIDDLE,SMALL,embedding
CODE,,,,
hb011,요식/유흥,요식,한식,0.178922
hb012,요식/유흥,요식,일식/중식/양식,0.212529
hb013,요식/유흥,요식,제과점,0.432291
hb014,요식/유흥,요식,커피전문점,0.439538
hb015,요식/유흥,요식,패스트푸드,0.372090


In [113]:
sim_list=[]
for i in range(len(df['SMALL'])):
    sim_list.append([df['SMALL'][i],ko_model.wv.similarity('유흥',df['MIDDLE'][i])])
print(sorted(sim_list, key=lambda x: x[1], reverse=True))

[['노래방', 1.0000001], ['기타유흥업소', 1.0000001], ['유흥주점', 1.0000001], ['영화/공연', 0.48998392], ['취미/오락', 0.48998392], ['서점', 0.48998392], ['호텔/콘도', 0.44355565], ['모텔/기타숙박', 0.44355565], ['종합레저시설', 0.42810345], ['여행사', 0.35401705], ['면세점', 0.35401705], ['한식', 0.35379973], ['일식/중식/양식', 0.35379973], ['제과점', 0.35379973], ['커피전문점', 0.35379973], ['패스트푸드', 0.35379973], ['미용서비스', 0.33305097], ['미용실', 0.33305097], ['학원', 0.32415295], ['종합병원', 0.32211018], ['일반병원', 0.32211018], ['치과병원', 0.32211018], ['한의원', 0.32211018], ['약국', 0.29729816], ['백화점', 0.2913504], ['할인점', 0.2913504], ['슈퍼마켓', 0.2913504], ['생활잡화', 0.2913504], ['편의점', 0.2913504], ['주유소', 0.28315607], ['LPG', 0.28315607], ['실내골프/헬스', 0.26825613], ['실외골프/스키', 0.26825613], ['스포츠시설', 0.26825613], ['생활서비스', 0.21869278], ['세탁소', 0.21869278], ['업무서비스', 0.21869278], ['패션/잡화', 0.1413931], ['안경', 0.1413931], ['귀금속', 0.1413931], ['화장품', 0.13640963], ['교육용품', 0.10670199], ['가전', 0.06369647], ['가구', 0.06369647], ['유아교육', 0.058876477], ['의복/의류', 0.04500903

In [60]:
print(ko_model.similarity(','유흥'));

AttributeError: 'FastText' object has no attribute 'similarity'

In [58]:
print(f"'아이언맨'과 '헐크'의 유사도: {m_fasttext.similarity('아이언맨', '헐크')}")
print(f"'아이언맨'과 '스파이더맨'의 유사도: {m_fasttext.similarity('아이언맨', '스파이더맨')}")

NameError: name 'm_fasttext' is not defined

In [54]:
for w, sim in ko_model.wv.similar_by_word('유흥업소', 10):
    print(f'{w}: {sim}')

유흥업소와: 0.7085636258125305
유흥업소의: 0.7007808089256287
유흥업소에: 0.6809409856796265
유흥업소나: 0.6766310930252075
유흥업소는: 0.6614196300506592
유흥업소에서: 0.6280500292778015
유흥업소가: 0.6276900172233582
유흥업소를: 0.6186465620994568
유흥업소로: 0.6177406311035156
유흥업소들이: 0.6114162802696228


In [44]:
#%
#임베딩 한 300차원의 벡터를 PCA 사용해서 2차원 벡터로 축소
from sklearn.decomposition import PCA

words = df['MIDDLE'].tolist()[:]
print(words)
pca = PCA(n_components=3)

xys = pca.fit_transform([ko_model.wv.word_vec(w) for w in words])
xs = xys[:,0]
ys = xys[:,1]
xy = xys[:,2]


['요식', '요식', '요식', '요식', '요식', '유흥', '유흥', '유흥', '유통', '유통', '유통', '유통', '유통', '음/식료품', '음/식료품', '음/식료품', '의복/의류', '패션/잡화', '패션/잡화', '패션/잡화', '스포츠', '스포츠', '스포츠', '레저', '문화', '문화', '문화', '숙박', '숙박', '여행', '여행', '미용', '미용', '화장품', '서비스', '서비스', '서비스', '학원', '유아교육', '교육용품', '병원', '병원', '병원', '병원', '약국', '가전/가구', '가전/가구', '자동차서비스/용품', '자동차서비스/용품', '주유', '주유']


c:\Users\pooh5\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  if __name__ == "__main__":


In [46]:
plt.figure(figsize=(15, 15))
plt.scatter(xs, ys,xy, marker='o',s=0.2)
for i, v in enumerate(words):
    plt.annotate(v, xy=(xs[i], ys[i]),fontsize=5)
    
df['embedding_x'] = xs
df['embedding_y'] = ys
# print(df)


TypeError: scatter() got multiple values for argument 's'

<Figure size 1500x1500 with 0 Axes>

array([0.80011859, 0.61146631, 0.49497298])

array([0.19913406, 0.15218215, 0.12318921])

In [50]:
#%
#카드끼리 묶음.

file_path = './minjeong.csv'
card_df = pd.read_csv(file_path, encoding='UTF8')
print(card_df.head())
print(len(card_df))
sorted_df = card_df.sort_values(by='거래시간')
sorted_df

   Unnamed: 0  Card His ID                 카드번호 category_id  거래지역    상점명  \
0           0          269  2568-9238-5337-7494       hb053  충청북도    남김권   
1           1          271  2568-9238-5337-7494       hb073  충청북도  (유) 최   
2           2          273  2568-9238-5337-7494       hb022  충청북도     류이   
3           3          275  2568-9238-5337-7494       hb016  충청북도     김이   
4           4          277  2568-9238-5337-7494       hb071  충청북도  (유) 이   

         거래날짜      거래시간   거래금액  
0  2023-04-22  19:02:00  98000  
1  2023-03-19  19:04:00  57000  
2  2023-05-04  19:28:00   6500  
3  2023-04-20  20:47:00  10000  
4  2023-05-31  17:06:00  25000  
600300


,Unnamed: 0,Card His ID,카드번호,category_id,거래지역,상점명,거래날짜,거래시간,거래금액
397563,397563,826775,4951-3534-2912-6880,hb025,대구광역시,유한회사 노박김,2023-04-19,00:00:00,4200
146746,146746,299185,5373-9217-2495-2579,hb025,경상남도,신이이,2023-05-06,00:00:00,26400
267162,267162,530297,7116-8267-8046-4505,hb025,충청북도,서류이,2023-05-18,00:00:00,18200
250106,250106,505685,3267-7510-3539-3420,hb025,강원도,유윤오,2023-03-27,00:00:00,4400
542589,542589,1111263,1436-3734-6761-4167,hb025,부산광역시,유한회사 박장민,2023-04-24,00:00:00,28900
...,...,...,...,...,...,...,...,...,...
158162,158162,322385,2443-7626-9132-9907,hb017,부산광역시,유한회사 박,2023-03-27,24:00:00,89000
371443,371443,758161,5868-1953-1431-7282,hb062,대전광역시,주식회사 김,2023-04-04,24:00:00,50000
536925,536925,1062041,9205-7427-6015-2494,hb062,제주특별자치도,박박홍,2023-03-30,24:00:00,80000
62454,62454,120449,1422-9238-7248-6133,hb062,세종특별자치시,주식회사 김김류,2023-05-17,24:00:00,100000


In [ ]:






#%
print(card_df['카드번호'].unique())
print(len(card_df['카드번호'].unique()))

card_IDs = card_df['카드번호'].unique()
#%
mean_card_df = {'카드번호': [], 'embedding_x_mean' : [], 'embedding_y_mean' : [], '거래금액': []}

for i, card_ID in tqdm.tqdm(enumerate(card_IDs)):
    _df = card_df[card_df['카드번호']==card_ID][['category_id', '거래금액']]
    # print(_df['category_id'].tolist())
    # print(df.head())
    # print(df.loc[['hb011','hb012']])
    # print(df.loc[_df['category_id']]['embedding_x'].mean())
    mean_card_df['카드번호'].append(card_ID)
    mean_card_df['embedding_x_mean'].append(df.loc[_df['category_id']]['embedding_x'].mean())
    mean_card_df['embedding_y_mean'].append(df.loc[_df['category_id']]['embedding_y'].mean())
    mean_card_df['거래금액'].append(_df['거래금액'].mean())
    # print(mean_card_df)

mean_card_df = pd.DataFrame(mean_card_df)
print(mean_card_df)
    # print(len(df))

# #%
# j = 0
# for i, card_ID in enumerate(card_df['카드번호']):
#     if card_df
# #%

#%
# 임베딩 벡터와 카드 번호 별로 저장
mean_card_df.to_csv('./SMALL_embedding_mean.csv') 


#%
# k-means
from sklearn.cluster import KMeans

n_clusters = 3
X = mean_card_df[['embedding_x_mean', 'embedding_y_mean', '거래금액']].values
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_
mean_card_df['Cluster'] = cluster_labels
print(mean_card_df)




In [ ]:
#%
#GMM
#여기서는 클러스터에 해당하는 각 카드의 결제내역을 모두 넣고 진행. 시간, 지역, 거래금액, 업종까지. 업종도 해시테이블 해야함(나중에 플랏 필요)
from sklearn.mixture import GaussianMixture

n_components = 3
GMM_columns = ['embedding_x_mean', 'embedding_y_mean', '거래금액']
gmm = GaussianMixture(n_components=n_components)

for i in range(n_clusters):
    X = mean_card_df[mean_card_df['Cluster'] == i][GMM_columns].values 
    gmm.fit(X)
    likelihoods = gmm.score_samples(X)
    plt.plot(likelihoods)
    plt.show()

    
#%